In [18]:
import pandas as pd
import numpy as np
import math

In [110]:
dfFromJson = pd.read_csv(r"C:\Users\rouma\Documents\BlueRock\Projekt\gpsData\gpsData\A9216\gpsSeriesA9216.csv")


## Specify time span

In [111]:
from datetime import datetime
startDate = '2021-07-23' #'2021-06-30'
endDate = '2021-07-24' # '2021-07-01'

#startDt = datetime.strptime(startDate, '%Y-%M-%d')
startDt = datetime.fromisoformat(startDate)
endDt = datetime.fromisoformat(endDate)

dateList = []
for d in dfFromJson['gpsAt']:
    dateList.append(pd.to_datetime(d))
dfFromJson['Date'] = dateList

In [112]:

dfJson_Small = dfFromJson[(dfFromJson['Date'] >= startDate) & (dfFromJson['Date'] < endDate)]
dfJson_Small = dfJson_Small.iloc[::-1] #Turn df upside down
dfJson_Small = dfJson_Small.reset_index()

In [113]:
# Get time difference between subsequent recordings #
time = [(t2-t1).total_seconds() /60 for t1, t2 in zip(dfJson_Small['Date'].shift(1), dfJson_Small['Date'])]

dfJson_Small['minDiff'] = time
splits = dfJson_Small[dfJson_Small['minDiff'] >= 60]['minDiff']
splitList = [0, max(dfJson_Small.index)]
for i in splits.items():
    print(i[0])
    splitList.append(i[0])


538
680
687


In [114]:
#[(dfJson_Small.index >= 8051) & (dfJson_Small.index <= 8058)]

dfList = []

for i in range(len(splitList)-1):
    print(splitList[i])
    dfList.append(dfJson_Small[(dfJson_Small.index >= splitList[i]) & (dfJson_Small.index < splitList[i+1])])

dfList[0]

0
689
538
680


,index,alt,batPct,gpsAt,heading,lat,lon,speed,Date,minDiff
0,8738,-3,100,2021-07-23T03:34:40Z,206,45.4824,-73.7197,0,2021-07-23 03:34:40+00:00,NaN
1,8737,21,100,2021-07-23T03:35:10Z,44,45.4829,-73.7176,14,2021-07-23 03:35:10+00:00,0.500000
2,8736,50,100,2021-07-23T03:35:22Z,312,45.4833,-73.7180,21,2021-07-23 03:35:22+00:00,0.200000
3,8735,37,100,2021-07-23T03:35:40Z,284,45.4840,-73.7193,20,2021-07-23 03:35:40+00:00,0.300000
4,8734,26,100,2021-07-23T03:35:49Z,206,45.4838,-73.7197,20,2021-07-23 03:35:49+00:00,0.150000
...,...,...,...,...,...,...,...,...,...,...
684,8054,157,92,2021-07-23T21:46:47Z,342,43.7051,-79.5817,20,2021-07-23 21:46:47+00:00,0.333333
685,8053,147,93,2021-07-23T21:47:18Z,260,43.7055,-79.5819,5,2021-07-23 21:47:18+00:00,0.516667
686,8052,173,93,2021-07-23T21:47:28Z,16,43.7057,-79.5820,0,2021-07-23 21:47:28+00:00,0.166667
687,8051,173,89,2021-07-23T22:48:58Z,16,43.7057,-79.5820,0,2021-07-23 22:48:58+00:00,61.500000


## Get duration of a segment
At first only interested in longer segments with a duration of over 3 hours

In [107]:
def getDurationOfSegment(df):
    tStart = dfList[0].iloc[0]['Date'].to_pydatetime()
    tEnd = dfList[0].iloc[-1]['Date'].to_pydatetime()
    return (tEnd - tStart).total_seconds() / 60

getDurationOfSegment(dfList[2])

346.1666666666667

In [117]:
# Select segments with sufficient duration
for i in dfList:
    if getDurationOfSegment(dfList[i]) < 180:
        dfList.pop(i)

,level_0,index,alt,batPct,gpsAt,heading,lat,lon,speed,Date,minDiff
0,0,8738,-3,100,2021-07-23T03:34:40Z,206,45.4824,-73.7197,0,2021-07-23 03:34:40+00:00,NaN
1,1,8737,21,100,2021-07-23T03:35:10Z,44,45.4829,-73.7176,14,2021-07-23 03:35:10+00:00,0.500000
2,2,8736,50,100,2021-07-23T03:35:22Z,312,45.4833,-73.7180,21,2021-07-23 03:35:22+00:00,0.200000
3,3,8735,37,100,2021-07-23T03:35:40Z,284,45.4840,-73.7193,20,2021-07-23 03:35:40+00:00,0.300000
4,4,8734,26,100,2021-07-23T03:35:49Z,206,45.4838,-73.7197,20,2021-07-23 03:35:49+00:00,0.150000
...,...,...,...,...,...,...,...,...,...,...,...
685,685,8053,147,93,2021-07-23T21:47:18Z,260,43.7055,-79.5819,5,2021-07-23 21:47:18+00:00,0.516667
686,686,8052,173,93,2021-07-23T21:47:28Z,16,43.7057,-79.5820,0,2021-07-23 21:47:28+00:00,0.166667
687,687,8051,173,89,2021-07-23T22:48:58Z,16,43.7057,-79.5820,0,2021-07-23 22:48:58+00:00,61.500000
688,688,8050,145,89,2021-07-23T22:49:18Z,124,43.7046,-79.5833,0,2021-07-23 22:49:18+00:00,0.333333
